<a href="https://colab.research.google.com/github/rezendervp/chemical-engineering/blob/main/Lattice_Boltzmann_2D_Cilynder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cmasher

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.1/367.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 4.6 MB/s eta 0:00:00


In [ ]:
"""
Solves the incompressible Navier Stokes equations using the Lattice-Boltzmann
Method¹. The scenario is the flow around a cylinder in 2D which yields a van
Karman vortex street.


                                periodic
        +-------------------------------------------------------------+
        |                                                             |
        | --->                                                        |
        |                                                             |
        | --->           ****                                         |
        |              ********                                       |
inflow  | --->        **********                                      |  outflow
        |              ********                                       |
        | --->           ****                                         |
        |                                                             |
        | --->                                                        |
        |                                                             |
        +-------------------------------------------------------------+
                                periodic

-> uniform inflow profile with only horizontal velocities at left boundary
-> outflow boundary at the right
-> top and bottom boundary connected by periodicity
-> the circle in the center (representing a slice from the 3d cylinder)
   uses a no-slip Boundary Condition
-> initially, fluid is NOT at rest and has the horizontal velocity profile
   all over the domain

¹ To be fully correct, LBM considers the compressible Navier-Stokes Equations.
This can also be seen by the fact that we have a changing macroscopic density over
the domain and that we actively use it throughout the computations. However, our
flow speeds are below the 0.3 Mach limit which results in only minor density
fluctuations. Hence, the fluid behaves almost incompressible.

------

Solution strategy:

Discretize the domain into a Cartesian mesh. Each grid vertex is associated
with 9 discrete velocities (D2Q9) and 2 macroscopic velocities. Then iterate
over time.


1. Apply outflow boundary condition on the right boundary

2. Compute Macroscopic Quantities (density and velocities)

3. Apply Inflow Profile by Zou/He Dirichlet Boundary Condition
   on the left boundary

4. Compute the discrete equilibria velocities

5. Perform a Collision step according to BGK (Bhatnagar–Gross–Krook)

6. Apply Bounce-Back Boundary Conditions on the cylinder obstacle

7. Stream alongside the lattice velocities

8. Advance in time (repeat the loop)


The 7th step implicitly yields the periodic Boundary Conditions at
the top and bottom boundary.

------

Employed Discretization:

D2Q9 grid, i.e. 2-dim space with 9 discrete
velocities per node. In Other words the 2d space is discretized into
N_x by N_y by 9 points.

    6   2   5
      \ | /
    3 - 0 - 1
      / | \
    7   4   8

Therefore we have the shapes:

- macroscopic velocity : (N_x, N_y, 2)
- discrete velocity    : (N_x, N_y, 9)
- density              : (N_x, N_y)


------

Lattice Boltzmann Computations

Density:

ρ = ∑ᵢ fᵢ


Velocities:

u = 1/ρ ∑ᵢ fᵢ cᵢ


Equilibrium:

fᵢᵉ = ρ Wᵢ (1 + 3 cᵢ ⋅ u + 9/2 (cᵢ ⋅ u)² − 3/2 ||u||₂²)


BGK Collision:

fᵢ ← fᵢ − ω (fᵢ − fᵢᵉ)


with the following quantities:

fᵢ  : Discrete velocities
fᵢᵉ : Equilibrium discrete velocities
ρ   : Density
∑ᵢ  : Summation over all discrete velocities
cᵢ  : Lattice Velocities
Wᵢ  : Lattice Weights
ω   : Relaxation factor

------

The flow configuration is defined using the Reynolds Number

Re = (U R) / ν

with:

Re : Reynolds Number
U  : Inflow Velocity
R  : Cylinder Radius
ν  : Kinematic Viscosity

Can be re-arranged in terms of the kinematic viscosity

ν = (U R) / Re

Then the relaxation factor is computed according to

ω = 1 / (3 ν + 0.5)

------

Note that this scheme can become unstable for Reynoldsnumbers >~ 350 ²

² Note that the stability of the D2Q9 scheme is mathematically not
linked to the Reynoldsnumber. Just use this as a reference. Stability
for this scheme is realted to the velocity magnitude.
Consequentially, the actual limiting factor is the Mach number (the
ratio between velocity magnitude and the speed of sound).

"""
#lyrabries and dependencies

import jax #for GPU e TPU fast calculations
import jax.numpy as jnp
import matplotlib.pyplot as plt
import cmasher as cmr
from tqdm import tqdm

#parameters
N_interations = 15_000  # the '_' improve the reading of big numbers
Reynolds_number = 80

N_points_X = 300
N_points_Y = 50

Cylinder_center_index_X = N_points_X // 5    #operator  // -> inter division
Cylinder_center_index_Y = N_points_Y // 2
Cylinder_radius_indices = N_points_Y // 9

Max_horizontal_inflow_velocity = 0.04

Visualize = True
Plot_every_N_steps = 25
Skip_first_N_iteractions = 0

#lattice parameters D2Q9

N_discrete_velocities = 9
"""
    6   2   5
      \ | /
    3 - 0 - 1
      / | \
    7   4   8
"""
#each vertical pair represents a direction acconding figure
lattice_velocities = jnp.array([
    [0,  1,  0, -1,  0,  1, -1, -1,  1,],
    [0,  0,  1,  0, -1,  1,  1, -1, -1,]
])

lattice_indices = jnp.array([
    0, 1, 2, 3, 4, 5, 6, 7, 8,
])

opposite_lattice_indices = jnp.array([
    0, 3, 4, 1, 2, 7, 8, 5, 6,
])

lattice_weights = jnp.array([
    4/9,                        #Center Velocity [0,]
    1/9, 1/9, 1/9, 1/9,        #Axis-aligned Velocities [1, 2, 3, 4]
    1/36, 1/36, 1/36, 1/36      #Diagonal-Aligned Velocities[5, 6, 7, 8,]
])

right_velocities = jnp.array([1, 5, 8])
up_velocities = jnp.array([2, 5, 6])
left_velocities = jnp.array([3, 6, 7])
down_velocities = jnp.array([4, 7, 8])
pure_vertical_velocities = jnp.array([0, 2, 4])
pure_horizontal_velocities = jnp.array([0, 1, 3])


#density computation

def get_density(discrete_velocities):
  density = jnp.sum(discrete_velocities, axis = -1)

  return density

# macroscopic velocities calculation (U,V)

def get_macroscopic_velocities(discrete_velocities, density):
  macroscopic_velocities = jnp.einsum(                      #Eistein Summation
      "NMQ,dQ->NMd",                                        #rule: N->X axis, M->Y axis, Q # of discrete velocities = 9, d-> macroscopic Axis =2
      discrete_velocities,                                  #vector/ matriz operated
      lattice_velocities,                                   #vector/ matriz operated
  ) / density[...,jnp.newaxis]
  return macroscopic_velocities

# equilibrium discrete velocities

def get_equilibrium_discrete_velocities(macroscopic_velocities, density):
  projected_discrete_velocities = jnp.einsum(
      "dQ, NMd->NMQ",
      lattice_velocities,
      macroscopic_velocities,
  )
  macroscopic_velocity_magnitude = jnp.linalg.norm(
      macroscopic_velocities,
      axis = -1,
      ord = 2,
  )
  equilibrium_discrete_velocities =(
      density[...,jnp.newaxis]
      *
      lattice_weights[jnp.newaxis, jnp.newaxis, :]
      *
      (
          1
          +
          3 * projected_discrete_velocities
          +
          9/2 * projected_discrete_velocities**2
          -
          3/2 * macroscopic_velocity_magnitude[...,jnp.newaxis]**2
      )
  )
  return equilibrium_discrete_velocities

def main():
  jax.config.update("jax_enable_x64", True)

  #viscosity calculation based on Re

  kinematic_viscosity = (
      Max_horizontal_inflow_velocity
      *
      Cylinder_radius_indices
  ) / (
      Reynolds_number
  )

  # LBM relaxation factor
  relaxation_omega = (
      (
       1.0
      ) / (
          3.0
          *
          kinematic_viscosity
          +
          0.5
      )
  )

  #define a Mesh

  x = jnp.arange(N_points_X)
  y = jnp.arange(N_points_Y)

  X, Y = jnp.meshgrid(x, y, indexing ="ij")

  #cylinder region:
  #obstacle mask: An array of the shpae like X or Y
  #contains True if the point belong to obstacle, and False if not
  obstacle_mask = (
      jnp.sqrt(
          (
              X
              -
              Cylinder_center_index_X
          )**2
          +
          (
              Y
              -
              Cylinder_center_index_Y
          )**2
      )
      <
          Cylinder_radius_indices
  )

  #velocity profile

  velocity_profile = jnp.zeros((N_points_X, N_points_Y, 2))
  velocity_profile = velocity_profile.at[:, :, 0].set(Max_horizontal_inflow_velocity)

  #SOLUTION ALGORITHM

  @jax.jit    #precompyle the function UPDATE

  def update(discrete_velocities_prev):
    #1) Precribe the outflow BC on the right boundary (Neumann BC)
    discrete_velocities_prev = discrete_velocities_prev.at[-1, :, left_velocities].set(
        discrete_velocities_prev[-2, :, left_velocities]
    )

    #2)Macroscopic Velocities
    density_prev = get_density(discrete_velocities_prev)
    macroscopic_velocities_prev = get_macroscopic_velocities(
        discrete_velocities_prev,
        density_prev,
    )

    #3) Prescribe Inflow Dirichlet unsing Zou-He Scheme
    macroscopic_velocities_prev =\
      macroscopic_velocities_prev.at[0, 1:-1, :].set(
          velocity_profile[0, 1:-1, :]
      )
    density_prev = density_prev.at[0, :].set(
        (
            get_density(discrete_velocities_prev[0, :, pure_vertical_velocities].T) # .T means Transposed
            +
            2 *
            get_density(discrete_velocities_prev[0, :, left_velocities].T)
        ) / (
            1 - macroscopic_velocities_prev[0, :, 0]
        )
    )
    #4) Compute Discrete Equilibrium Velocities
    equilibrium_discrete_velocities = get_equilibrium_discrete_velocities(
        macroscopic_velocities_prev,
        density_prev,
    )

    #3.1 Belongs to Zou-He Scheme
    discrete_velocities_prev =\
      discrete_velocities_prev.at[0, :, right_velocities].set(
          equilibrium_discrete_velocities[0, :, right_velocities]
      )

    #5) Collisions according to BGK approach
    discrete_velocities_post_colision = (
        discrete_velocities_prev
        -
        relaxation_omega
        *
        (
            discrete_velocities_prev
            -
            equilibrium_discrete_velocities
        )
    )

    #6) Bounce-back BC to enforce th no-slip on Cylinder
    for i in range(N_discrete_velocities):
      discrete_velocities_post_colision =\
        discrete_velocities_post_colision.at[obstacle_mask, lattice_indices[i]].set(
            discrete_velocities_prev[obstacle_mask, opposite_lattice_indices[i]]
            #it must there is an inversion of veloticies directions!
        )

    #7) Stream alonside lattice velocities with periodic BC implicity
    discrete_velocities_streamed = discrete_velocities_post_colision
    for i in range(N_discrete_velocities):
      discrete_velocities_streamed = discrete_velocities_streamed.at[:, :, i].set(
           jnp.roll(
              jnp.roll(
                  discrete_velocities_post_colision[:, : , i],
                  lattice_velocities[0, i],
                  axis = 0
              ),
              lattice_velocities[1, i],
              axis = 1,
           )
      )
    return discrete_velocities_streamed

   #Initialization of velocities
  discrete_velocities_prev = get_equilibrium_discrete_velocities(
      velocity_profile,
      jnp.ones((N_points_X, N_points_Y))
  )

  #defining global graphic styles
  plt.style.use("dark_background")
  #plt.style.use("default")
  plt.rcParams.update({'font.size': 2.5})

  #Time Iteraction
  for iteration_index in tqdm(range(N_interations)):
    discrete_velocities_next = update(discrete_velocities_prev)

    discrete_velocities_prev = discrete_velocities_next

    #Show plots
                       # IF exact division....
    if iteration_index % Plot_every_N_steps == 0 and Visualize and iteration_index > Skip_first_N_iteractions:
      # densities
      density = get_density(discrete_velocities_next)


      #macroscopic velocities U,V
      macroscopic_velocities = get_macroscopic_velocities(
          discrete_velocities_next,
          density,
      )
      #Macroscopi velocity magnitude
      velocity_magnitude = jnp.linalg.norm(
          macroscopic_velocities,
          axis = -1,
          ord = 2,
      )
      #derivatives of velocity vector components
      du_dx, du_dy = jnp.gradient(macroscopic_velocities[..., 0])  #X component
      dv_dx, dv_dy = jnp.gradient(macroscopic_velocities[..., 1])  #Y component

      U  = macroscopic_velocities[..., 0]
      V  = macroscopic_velocities[..., 1]



      # curl of velocity
      curl = (du_dy - dv_dx)

      #Figure Size adn resolution
      plt.figure(figsize=(3,3), dpi =300)

      #Density Field
      plt.subplot(411)
      plt.contourf(
          X,
          Y,
          density,
          levels = 51,
          cmap = cmr.chroma,
      )

      plt.colorbar().set_label("Density")
      plt.gca().add_patch(plt.Circle(
          (Cylinder_center_index_X, Cylinder_center_index_Y),
          Cylinder_radius_indices,
          color ="black"
      )
      )

      #skiping  mesh and vectors arrays - better to see the vetor field
      Xsub = X[::10]
      Ysub = Y[::10]
      Usub = U[::10]
      Vsub = V[::10]

      #Vector Field
      plt.subplot(412)
      plt.quiver(
          Xsub,
          Ysub,
          Usub,
          Vsub,
          color = "dimgray",
          scale= 2
      )
      plt.gca().add_patch(plt.Circle(
          (Cylinder_center_index_X, Cylinder_center_index_Y),
          Cylinder_radius_indices,
          color ="black"
      )
      )

      #Velocity Magnitude
      plt.subplot(413)
      plt.contourf(
          X,
          Y,
          velocity_magnitude,
          levels = 51,
          cmap = cmr.amber,
      )
      plt.colorbar().set_label("Velocity Magnitude")
      plt.gca().add_patch(plt.Circle(
          (Cylinder_center_index_X, Cylinder_center_index_Y),
          Cylinder_radius_indices,
          color ="black"
      )
      )

      #Vorticity Magnitude Contour Plot in the botton
      plt.subplot(414)
      plt.contourf(
          X,
          Y,
          curl,
          levels = 51,
          cmap = cmr.redshift,
          vmin = -0.02,
          vmax =  0.02,
      )
      plt.colorbar().set_label("Vorticity Magnitude")
      plt.gca().add_patch(plt.Circle(
          (Cylinder_center_index_X, Cylinder_center_index_Y),
          Cylinder_radius_indices,
          color ="black"
      )
      )
      #plt.tight_layout(pad=5.0)
      plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.9,
                    top=0.9,
                    wspace=0.4,
                    hspace=0.4)
      plt.draw()
      plt.savefig(f'/content/drive/MyDrive/Colab Notebooks/lattice boltzmann figures/cylinder{iteration_index}.jpg', format='jpg')
      plt.pause(0.0001)
      plt.clf()



  if Visualize:
    plt.show()



if __name__== "__main__":
  main()

